In [1]:
'''
Import packages necessary to run script
'''
# def import_packages():
# import tkinter as tk
# from tkinter import filedialog, simpledialog, messagebox
import math
# from statistics import mean, stdev
import numpy as np
import pandas as pd
import statistics
# import os
# %matplotlib inline
# from datetime import datetime
# import matplotlib.pyplot as plt
# import scipy.stats as sps
# import sys
# import scipy.optimize as spo
# return print('packages imported')
# import_packages()

In [2]:
'''
Fetch working directory, and set modules directory
'''
import os, sys
cwd = os.getcwd()
modules_path = cwd + '\Modules\\'
sys.path.insert(1, modules_path)
print(f"Added {modules_path} as modules folder")

Added C:\Users\Matt\OneDrive - University of Toronto\Scripts\Revised Script - Feb 2022\Modules\ as modules folder


In [3]:
'''
Set up input and output folders and scan settings
'''
# def initial_setup(addfolderswithdate, getinput_type):
def initial_setup(addfolderswithdate):
    import InitialSetup as InitialSetup
    input_output_folders = InitialSetup.get_folders(withdate=addfolderswithdate)
    # settings = InitialSetup.get_data_type(get_input=True)
    # return (input_output_folders, settings)
    return (input_output_folders)

addfolderswithdate, getinput_type = False, True #change this to make new folder with date
addfolderswithdate = True
initial_setup_output = initial_setup(addfolderswithdate)
file_path_input = initial_setup_output[0]
file_path_output = initial_setup_output[1]
print(f"Copy input files to /{file_path_input}")

Copy input files to /Input_Data_2023-03-28


In [4]:
'''
Function for retrieving csv file and returning in indicated format
'''
def get_csv(return_type, title, initialdir, skip_blank_lines, skiprows):
    import tkinter as tk
    from tkinter import filedialog
    import GeneralFunctions as GeneralFunctions
    csvfile = tk.filedialog.askopenfilename(title=title, initialdir=initialdir, filetype=(("CSV files", ".csv"), ("all files", "*.*")))
    file = open(csvfile)
    if return_type == 'lines':
        file_lines = GeneralFunctions.get_file_lines(file)
        file.close()
        return(file_lines)
    elif return_type == 'df' :
        file_df = pd.read_csv(file, skip_blank_lines=skip_blank_lines, skiprows=skiprows)
        file.close()
        return(file_df)
    elif return_type == 'both':
        file_df = pd.read_csv(file, skip_blank_lines=skip_blank_lines, skiprows=skiprows)
        file.close()
        file = open(csvfile)
        file_lines = GeneralFunctions.get_file_lines(file)
        file.close()
        return(file_lines, file_df)
print("defined function for retrieving csv files")

defined function for retrieving csv files


In [5]:
'''
Fetch profile in both formats
'''
# profile_lines, profile_df = get_csv(return_type='both', title='Profile', initialdir=file_path_input)
# profile_df = get_csv(return_type='df', title='Profile', initialdir=file_path_input, skip_blank_lines=False, skiprows=None)
input_df = get_csv(return_type='df', title='Input', initialdir=file_path_input, skip_blank_lines=False, skiprows=1)
# can change the return type to lines once I add legacy functionality back in
print("profile file imported")

defined function for exporting aligned peak table as csv
profile file imported


In [6]:
input_df.columns

Index(['D:\Work\Brock\Data files\170629-WT-SA-1-01.D ', 'Peak ', 'Start', 'RT',
       'End', 'Height', 'Area', 'Area %', 'Height %', 'Area Sum %', 'Width',
       'FWHM', 'Start Y', 'End Y', 'B1 Start', 'B1 End', 'B2 Start', 'B2 End',
       'Start BL Y', 'End BL Y', 'Symmetry', 'Max Y', 'SNR', 'Height % (Norm)',
       'Baseline', 'Base Peak', 'k'', 'Label', 'Cpd', 'Type', 'Plates',
       'Plates/M', 'Resolution', 'Saturated', 'Tailing factor', 'Flags (Tgt)',
       'RI', 'Unnamed: 37'],
      dtype='object')

In [7]:
try:
    del trimmed_df
except:
    print("no df to remove from memory")
trimmed_df = input_df[['Start', 'RT', 'End','Base Peak']].copy()
trimmed_df['SampleFile'] = input_df.iloc[:,0]

trimmed_df['RT'] = pd.to_numeric(trimmed_df['RT'], errors='coerce')
trimmed_df = trimmed_df[trimmed_df['RT'] > 0]
trimmed_df['Start'] = pd.to_numeric(trimmed_df['Start'], errors='coerce')
trimmed_df['End'] = pd.to_numeric(trimmed_df['End'], errors='coerce')
trimmed_df = trimmed_df.sort_values(by=['RT', 'Start'])
trimmed_df

no df to remove from memory


,Start,RT,End,Base Peak,SampleFile
1317,4.609,4.631,4.731,171.1,D:\Work\Brock\Data files\170704-TGA256-SA-3-03.D
3062,4.609,4.631,4.720,171.1,D:\Work\Brock\Data files\170710-TGA256-4-03.D
579,4.609,4.632,4.721,171.1,D:\Work\Brock\Data files\170629-WT-SA-4-03.D
801,4.609,4.632,4.687,171.1,D:\Work\Brock\Data files\170704-Control-No-Tis...
830,4.609,4.632,4.732,171.1,D:\Work\Brock\Data files\170704-TGA256-SA-1-01.D
...,...,...,...,...,...
256,51.204,53.370,54.515,207.1,D:\Work\Brock\Data files\170629-WT-SA-2-02.D
98,50.126,53.371,55.326,207.1,D:\Work\Brock\Data files\170629-WT-SA-1-02.D
360,50.171,53.393,55.059,207.1,D:\Work\Brock\Data files\170629-WT-SA-3-01.D
2459,51.626,54.359,54.681,207.1,D:\Work\Brock\Data files\170707-WT-6-01.D


In [8]:
try:
    del features_df
except:
    print("no df to remove from memory")
# x = trimmed_df.iloc[0,:]
features_df = pd.DataFrame([trimmed_df.iloc[0]],columns=trimmed_df.columns)
features_df['RTlist'] = [[]]
features_df['BPlist'] = [[]]
features_df['SampleList'] = [[]]
features_df.head()

no df to remove from memory


,Start,RT,End,Base Peak,SampleFile,RTlist,BPlist,SampleList
1317,4.609,4.631,4.731,171.1,D:\Work\Brock\Data files\170704-TGA256-SA-3-03.D,[],[],[]


In [9]:
previous_feature = features_df.iloc[-1].copy()
for index, row in trimmed_df.iterrows():
    feature = row.copy()
    previous_start = previous_feature['Start']
    previous_end = previous_feature['End']
    RT = feature['RT']
#     print(previous_start, RT, previous_end)
    if previous_start < RT < previous_end:
        SampleFile = feature['SampleFile']
        if SampleFile not in previous_feature['SampleList']:
            previous_feature['RTlist'].append(RT)
            previous_feature['BPlist'].append(feature['Base Peak'])
            previous_feature['SampleList'].append(SampleFile)
            previous_feature['Start'] = min([feature['Start'],previous_start])
            previous_feature['End'] = max([feature['End'],previous_end])
            # print(previous_feature)
    else:
        RT = statistics.median(previous_feature['RTlist'])
        BP = statistics.mode(previous_feature['BPlist'])
        previous_feature['RT'] = RT
        previous_feature['Base Peak'] = BP
        features_df.iloc[-1] = previous_feature
        feature['RTlist'] = [(feature['RT'])]
        feature['BPlist'] = [(feature['Base Peak'])]
        feature['SampleList'] = [(feature['SampleFile'])]
        features_df = pd.concat([features_df, feature.to_frame().T], ignore_index=True)
        previous_feature = feature
        
features_df.head()


,Start,RT,End,Base Peak,SampleFile,RTlist,BPlist,SampleList
0,4.609,4.632,4.732,171.1,D:\Work\Brock\Data files\170704-TGA256-SA-3-03.D,"[4.631, 4.631, 4.632, 4.632, 4.632, 4.632, 4.6...","[171.1, 171.1, 171.1, 171.1, 171.1, 171.1, 171...",[D:\Work\Brock\Data files\170704-TGA256-SA-3-0...
1,4.687,4.776,4.809,79.1,D:\Work\Brock\Data files\170704-Control-No-Tis...,[4.776],[79.1],[D:\Work\Brock\Data files\170704-Control-No-Ti...
2,4.809,4.832,4.865,79.1,D:\Work\Brock\Data files\170704-Control-No-Tis...,[4.832],[79.1],[D:\Work\Brock\Data files\170704-Control-No-Ti...
3,4.82,4.943,5.098,73.1,D:\Work\Brock\Data files\170710-TGA256-1-03.D,"[4.92, 4.92, 4.92, 4.92, 4.92, 4.921, 4.921, 4...","[73.1, 73.1, 73.1, 73.1, 73.1, 73.1, 73.1, 73....",[D:\Work\Brock\Data files\170710-TGA256-1-03.D...
4,5.098,5.143,5.254,159.1,D:\Work\Brock\Data files\170704-Control-No-Tis...,[5.143],[159.1],[D:\Work\Brock\Data files\170704-Control-No-Ti...


In [10]:
profile_df = features_df[['RT', 'RT', 'Start','End']].copy()
profile_df.columns.values[0] = 'Peak'
profile_df.head()

,Peak,RT,Start,End
0,4.632,4.632,4.609,4.732
1,4.776,4.776,4.687,4.809
2,4.832,4.832,4.809,4.865
3,4.943,4.943,4.82,5.098
4,5.143,5.143,5.098,5.254


In [11]:
'''
Format and Display the first 5 lines of the profile dataframe
'''

def rename_duplicates(input_series):
    renamed_series = input_series
    series_list = []
    for item in input_series:
        counter = 1
        new_item = item
        while new_item in series_list:
            counter += 1
            new_item = item + '_' + str(counter)
        series_list.append(new_item)
    renamed_series_values = pd.Series(series_list)
    return renamed_series_values

profile_df['Peak'] = rename_duplicates(profile_df['Peak'])
profile_df.head()

,Peak,RT,Start,End
0,4.632,4.632,4.609,4.732
1,4.776,4.776,4.687,4.809
2,4.832,4.832,4.809,4.865
3,4.943,4.943,4.82,5.098
4,5.143,5.143,5.098,5.254


In [12]:
profile_df['Peak']

0      4.632
1      4.776
2      4.832
3      4.943
4      5.143
       ...  
86    45.227
87    45.816
88    46.060
89    48.927
90    51.026
Name: Peak, Length: 91, dtype: float64

In [13]:
def get_areas_df(file_path_input):
    print("fetching areas")
    areas_df = get_csv(return_type='df', title='Areas', initialdir=file_path_input, skip_blank_lines=True, skiprows=1)
    areas_df.columns.values[0] = 'SampleID'
    areas_df.head()
    return areas_df

In [14]:
def get_sampleID_from_area(sample_file):
    ## works to return sample ID from full info line
    ## #"+CI Scan (rt: 4.762 min) 210505_NH3-pos-split-scan_361-R2.D  Subtract "
    start_character = "\\"
    end_character = ".D"
    start_index = sample_file.rfind(start_character) + len(start_character)
    end_index = sample_file.rfind(end_character)
    sampleID = sample_file[start_index:end_index]
    return sampleID

In [15]:
def get_area_tables(profile_df, areas_df):
    print("Separating features by sample")
    areas_df_split = [y for x, y in areas_df.groupby('SampleID', as_index=False)] # Split dataframe by SampleID

    profile_headers = profile_df.columns.tolist()
    extra_headers = ['sampleID', 'in_profile', 'flag','RT_difference']
    area_headers = areas_df.columns.tolist()
    feature_headers = extra_headers + area_headers
    columnheaders = profile_headers + extra_headers + area_headers
    all_rows = [columnheaders]

    simplified_headers = profile_headers
    all_simplified_features = []

    first_run_for_sample = True
    for index1,profilefeature in profile_df.iterrows():
        profileID, profileRT, lowerrt, upperrt = profilefeature['Peak'], profilefeature['RT'], profilefeature['Start'], profilefeature['End']
#         print(f"Working on: {profileID}")
        profile_line = profilefeature.tolist()
        this_simplified_feature = profilefeature.tolist()
        this_feature = []
        flag, previous_sample, first_run_for_feature = None, None, True
        sample_counter = 0 
        for sample_df in areas_df_split:
            sample_df['RT'] = pd.to_numeric(sample_df['RT'], errors='coerce') # Sets non numeric as NaN
            sample_df = sample_df[sample_df['RT'] > 0]
            if len(sample_df) > 0: # Only process data files and lines with at least 1 features
                sample_file = sample_df['SampleID'].iloc[0]
                sampleID = get_sampleID_from_area(sample_file)                 
#                 print(f"processing: {sampleID}")
                if first_run_for_sample:
                    simplified_headers.append(sampleID)
                in_profile=False
                possible_features = []

    #                 rt_index = sample_df.columns.get_loc("RT")
    #                 area_index = sample_df.columns.get_loc("Area")
                for index1,feature in sample_df.iterrows():
                    this_feature = []
                    rt,area = feature['RT'],feature['Area']
                    if lowerrt < rt < upperrt: # feature rt is within profile rt bounds
                        # feature confirmed in profile
                        flag = None
                        in_profile = True
                        RT_difference = rt - profileRT
                        RT_difference = round(RT_difference,4)
                        this_feature=[sampleID, in_profile, flag, RT_difference]
                        feature_values = feature.values.tolist()
                        this_feature = this_feature + feature_values
                        possible_features.append(this_feature)

                if in_profile:
                    sample_counter += 1
                    possible_features_df = pd.DataFrame(possible_features,columns=feature_headers)
                    this_feature_df = possible_features_df[possible_features_df['RT_difference'] == possible_features_df['RT_difference'].min()]
                    this_feature_df = this_feature_df.loc[0]
                    area = this_feature_df['Area']
                    this_feature = this_feature_df.tolist()
                    this_row = profile_line + this_feature
                    all_rows.append(this_row)
                    this_simplified_feature.append(area)
    #                 print(f"{profileID} added for {sampleID}")
                else:
                    this_simplified_feature.append(None)
    #                 print(f"{profileID} not found for {sampleID}")
        all_simplified_features.append(this_simplified_feature)
        first_run_for_sample = False
        print(f"{profileID} found in {sample_counter} samples.")
            
    #         print(f"{profileID} completed")
#     simplified_area_table = simplified_headers + all_simplified_features
    detailed_areas_df = pd.DataFrame(all_rows,columns=columnheaders)
    simplified_areas_df = pd.DataFrame(all_simplified_features,columns=simplified_headers)
    return(detailed_areas_df, simplified_areas_df)




In [16]:
def process_areas(file_path_input, file_path_output):
    get_areas = True
    if get_areas is True:
        areas_df = get_areas_df(file_path_input)
        print("Processing areas data table...")
        detailed_areas_df, simplified_areas_df = get_area_tables(profile_df, areas_df)
        print("Dataframe for all features with details complete")
        output_file_name1 = 'detailed_areas_df.csv'
        output_file_name2 = 'simplified_areas_df.csv'
        print(f"Exporting {output_file_name1} and {output_file_name2} to {file_path_output}")
        output_file_name_path1 = os.path.join(file_path_output, output_file_name1)
        output_file_name_path2 = os.path.join(file_path_output, output_file_name2)
        try:
            detailed_areas_df.to_csv(output_file_name_path1, sep=',', na_rep='', index=False, header=False)
            print("Export complete")
        except:
            print(f"Error in exporting {output_file_name1} to {output_file_name_path1}.\nMake sure a file with this name is not already open.\nContinuing without exporting this file.")
        try:
            simplified_areas_df.to_csv(output_file_name_path2, sep=',', na_rep='', index=False, header=True)
            print("Export complete")
        except:
            print(f"Error in exporting {output_file_name2} to {output_file_name_path2}.\nMake sure a file with this name is not already open.\nContinuing without exporting this file.")

        simplified_areas_df.head()
    else:
        quit()
        
process_areas(file_path_input, file_path_output)
    

fetching areas
Processing areas data table...
Separating features by sample
4.632 found in 40 samples.
4.776 found in 1 samples.
4.832 found in 1 samples.
4.943 found in 52 samples.
5.143 found in 1 samples.
5.987 found in 1 samples.
6.387 found in 1 samples.
6.498 found in 25 samples.
7.465 found in 1 samples.
7.754 found in 1 samples.
8.376 found in 65 samples.
9.076 found in 23 samples.
10.003499999999999 found in 6 samples.
10.909 found in 4 samples.
11.309 found in 39 samples.
11.686 found in 43 samples.
11.786 found in 65 samples.
12.009 found in 1 samples.
12.264 found in 3 samples.
12.82 found in 64 samples.
13.231 found in 41 samples.
13.819 found in 65 samples.
14.008 found in 1 samples.
14.264 found in 62 samples.
15.264 found in 65 samples.
16.33 found in 1 samples.
16.741 found in 2 samples.
16.997 found in 42 samples.
17.219 found in 65 samples.
17.897 found in 64 samples.
18.241 found in 23 samples.
18.963 found in 64 samples.
19.186 found in 28 samples.
19.63 found in 3

In [17]:
quit()